### Install Dependencies 

In [ ]:
!pip install tokenizers
!pip install transformers[torch]
!pip install accelerate

### KILL CODE

In [ ]:
# import os
# os._exit(00)

### CHECK TO MAKE SURE YOU GOT A GPU

In [2]:
import torch
torch.cuda.is_available()

True

#### VERSION CHECK

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

### Connect to GDrive 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Lets make some folders

In [ ]:
# !mkdir -p /content/drive/MyDrive/models/xmas/tokenizer

Converting Your CSV to JSON

In [ ]:
import csv
import json
def convert_csv_to_json(csv_file_path):
    # Read CSV file
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    # Convert CSV data to JSON
    json_data = json.dumps(rows, indent=4)

    # Save JSON data to a file (optional)
    with open('../data/All_Playlists_Combined.json', 'w') as json_file:
        json_file.write(json_data)

    return json_data

# Specify the path to your CSV file
csv_file_path = '../data/All Playlists Combined.csv'

# Convert CSV to JSON
json_data = convert_csv_to_json(csv_file_path)

print("Conversion completed. JSON data:")
print(json_data)

Helper Functions to normalize your data

In [17]:
import re
def remove_special_characters_and_spaces(input_string):
    # Define a regular expression pattern to match special characters and spaces
    pattern = r'[^a-zA-Z0-9]+'  # This pattern will keep only letters and digits

    # Use the sub method to replace matches of the pattern with an empty string
    clean_string = re.sub(pattern, '', input_string)

    return clean_string

def remove_special_characters(input_string):
    # Define a regular expression pattern to match special characters and spaces
    pattern = r'[^a-zA-Z0-9\s]+'  # This pattern will keep only letters and digits

    # Use the sub method to replace matches of the pattern with an empty string
    clean_string = re.sub(pattern, '', input_string)

    return clean_string

### Proto Prompting 

In [ ]:
import csv
import json

In [2]:
def convert_csv_to_json(csv_file_path):
    # Read CSV file
    with open(csv_file_path, 'r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)
    # Convert CSV data to JSON
    json_data = json.dumps(rows, indent=4)

    # Save JSON data to a file (optional)
    with open(f'{csv_file_path}.json', 'w') as json_file:
        json_file.write(json_data)

    return json_data

In [40]:



# Specify the path to your CSV file
csv_file_path = '../data/All Playlists Combined.csv'

# Convert CSV to JSON
json_data = convert_csv_to_json(csv_file_path)

print("Conversion completed. JSON data:")
# print(json_data)
json_data = json.loads(json_data)
songs = {}
songs['train']={
    'text':[],
    'id':[]
}
songs['validation']={
    'text':[],
    'id':[]
}
songs['test']={
    'text':[],
    'id':[]
}
count = 1
for i in json_data:
    songs['train']['id'].append(f"train-{count}")
    songs['train']['text'].append(f"""Given a target song title, write a christmas song that corresponds to the target song title.


### Target song title:
{i["track_name"]}


### Song Lyrics:
{i["lyrics"]}
"""}
    songs['validation']['id'].append(f"validation-{count}")
    songs['validation']['text'].append(f"""Given a target song title, write a christmas song that corresponds to the target song title.


### Target song title:
{i["track_name"]}


### Song Lyrics:
""")
    songs['test']['id'].append(f"test-{count}")
    songs['test']['text'].append(f"""Given a target song title, write a christmas song that corresponds to the target song title.


### Target song title:
{i["track_name"]}


### Song Lyrics:
{i["lyrics"]}
""")
    count+=1

with open(f'../data/songs.json', 'w', encoding='utf-8') as f:
    json.dump(songs, f, ensure_ascii=True, indent=4)
    f.close()          

Conversion completed. JSON data:


#### Fine-tuning a Mixtral-7b 

In [42]:
from datasets import load_dataset

train_dataset = load_dataset("json", data_files='../data/songs.json',field="train", split='train')
eval_dataset = load_dataset("json", data_files='../data/songs.json', split='validation') 
test_dataset = load_dataset("json", data_files='../data/songs.json', split='test') 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

Given a target song title, write a christmas song that corresponds to the target song title.


### Target song title:
Rockin' Around The Christmas Tree


### Song Lyrics:


[Chorus]
Rockin' around the Christmas tree
At the Christmas party hop
Mistletoe hung where you can see
Ev'ry couple tries to stop

Rockin' around the Christmas tree
Let the Christmas spirit ring
Later we'll have some pumpkin pie
And we'll do some caroling

[Bridge]
You will get a sentimental feeling when you hear
Voices singing "Let's be jolly -
Deck the halls with boughs of holly!"

[Chorus]
Rockin' around the Christmas tree
Have a happy holiday
Everyone's dancing merrily
In a new old fashioned way

[Bridge]
You will get a sentimental feeling when you hear
Voices singing "Let's be jolly -
Deck the halls with boughs of holly!"

[Chorus]
Rockin' around the Christmas tree
Have a happy holiday
Everyone's dancing merrily
In a new old fashioned way





In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


base_model_id = "../../Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""Given a target song title, write a christmas song that corresponds to the target song title.


### Target song title:
{data_point["target"]}


### Song Lyrics:
{data_point["lyrics"]}
"""
    return tokenize(full_prompt)

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

In [ ]:
print(tokenized_train_dataset[4]['input_ids'])

In [ ]:
print("Target song title: " + test_dataset[1]['target'])
print("Song Lyrics: " + test_dataset[1]['lyrics'] + "\n")

In [ ]:
eval_prompt = f"""Given a target song title, provide the lyrics of song that corresponds to the target song title.


### Target song title:
    Rockin' Around The Christmas Tree


### Song Lyrics:
"""

In [ ]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))